In [2]:
import numpy as np

In [ ]:
# The parameters the paper said
print(1 + (1/2 + 4/5) * 1/50)
print((1.026)**50)

1.026
3.608843415268567


In [ ]:
# The parameters the paper actually used
print(1.34865**(1/50))
print(1.006**50)
print(0.006*50) # gamma + beta

1.0060000102257765
1.3486493145631329
0.3


In [ ]:
# precision scaling
print(100**(1/2))

10.0


In [7]:
x = 0
rho = (4-x**2)**(1/2) / (2 * np.pi)
print(rho)

np.random.normal(0, 1)

0.3183098861837907


0.9617375094946492